In [3]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Acre - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [9]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Acre - Produção de Cimento (t)', axis=1)
data

,Ano,Acre - IDH Renda,Acre - IDH,Acre - PIB Estadual,Acre - PIB Construção Civil,Acre - PIB Per Capita,Acre - PIB Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - IDH Longevidade,Acre - Consumo de Cimento (t)
0,2014,0.660516,0.711115,8.560896e+06,400704.121188,9.343267,6.852462e+06,5.676413e+08,2.779340e+09,1.954361e+10,...,23.987188,985.694959,1230.934473,26.457083,21.165984,76.068571,1.660711,11.666939,0.796188,171.709286
1,2015,0.663153,0.720255,8.597116e+06,391942.316578,9.284504,6.825760e+06,5.179392e+08,2.561731e+09,1.327596e+10,...,28.288853,986.021421,1278.252041,26.043363,21.489384,78.704494,1.603219,12.139670,0.808026,151.486629
2,2016,0.662167,0.720460,8.620855e+06,389049.090551,9.247193,6.809907e+06,4.524144e+08,2.516859e+09,1.011370e+10,...,31.329110,985.952489,1333.799567,25.936106,21.376188,78.087996,1.551043,12.458072,0.807937,134.223086
3,2017,0.662364,0.720230,8.632052e+06,389052.079088,9.235469,6.808242e+06,4.213689e+08,2.497710e+09,8.456309e+09,...,32.173955,987.415055,1385.250388,26.527832,21.280819,74.803746,1.421881,12.245790,0.807295,150.491000
4,2018,0.663033,0.720978,8.619421e+06,391132.683164,9.229845,6.804838e+06,4.166580e+08,2.797145e+09,7.937760e+09,...,33.981418,990.300474,1358.141602,27.126298,21.732628,74.092935,1.192635,12.148886,0.806140,148.696000
5,2019,0.661854,0.719885,8.563064e+06,393404.510680,9.229435,6.792501e+06,4.462625e+08,3.484252e+09,1.091282e+10,...,36.963600,991.748979,1319.203565,27.958921,22.407138,73.297486,0.958636,12.045733,0.804426,153.806000
6,2020,0.661296,0.718147,8.474525e+06,395956.006706,9.239313,6.780171e+06,5.489412e+08,4.114911e+09,1.625333e+10,...,34.795058,990.943430,1329.282952,29.233325,22.624746,69.064652,0.790134,11.905311,0.802101,178.033000
7,2021,0.660686,0.715615,8.328315e+06,399296.137980,9.262962,6.763503e+06,6.638811e+08,4.421848e+09,1.933031e+10,...,31.757862,990.216809,1380.754715,29.633694,22.179792,66.100954,0.580550,11.817334,0.800019,189.357000
8,2022,0.661203,0.716125,8.150141e+06,403056.753694,9.289289,6.741052e+06,7.010332e+08,4.533216e+09,1.997888e+10,...,42.542791,990.175184,1380.835509,29.449893,22.067993,66.491567,0.541832,11.830301,0.801112,202.321000


In [10]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Acre - IDH Renda,Acre - IDH,Acre - PIB Estadual,Acre - PIB Construção Civil,Acre - PIB Per Capita,Acre - PIB Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Acre - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - IDH Longevidade
0,0.660516,0.711115,8.560896e+06,400704.121188,9.343267,6.852462e+06,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,0.683294,23.987188,985.694959,1230.934473,26.457083,21.165984,76.068571,1.660711,11.666939,0.796188
1,0.663153,0.720255,8.597116e+06,391942.316578,9.284504,6.825760e+06,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,0.697137,28.288853,986.021421,1278.252041,26.043363,21.489384,78.704494,1.603219,12.139670,0.808026
2,0.662167,0.720460,8.620855e+06,389049.090551,9.247193,6.809907e+06,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,0.698785,31.329110,985.952489,1333.799567,25.936106,21.376188,78.087996,1.551043,12.458072,0.807937
3,0.662364,0.720230,8.632052e+06,389052.079088,9.235469,6.808242e+06,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,0.698311,32.173955,987.415055,1385.250388,26.527832,21.280819,74.803746,1.421881,12.245790,0.807295
4,0.663033,0.720978,8.619421e+06,391132.683164,9.229845,6.804838e+06,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,...,0.700794,33.981418,990.300474,1358.141602,27.126298,21.732628,74.092935,1.192635,12.148886,0.806140
5,0.661854,0.719885,8.563064e+06,393404.510680,9.229435,6.792501e+06,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,...,0.700321,36.963600,991.748979,1319.203565,27.958921,22.407138,73.297486,0.958636,12.045733,0.804426
6,0.661296,0.718147,8.474525e+06,395956.006706,9.239313,6.780171e+06,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,...,0.697805,34.795058,990.943430,1329.282952,29.233325,22.624746,69.064652,0.790134,11.905311,0.802101


In [11]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    151.486629
1    134.223086
2    150.491000
3    148.696000
4    153.806000
5    178.033000
6    189.357000
7    202.321000
8           NaN
Name: Acre - Consumo de Cimento (t), dtype: float64

In [18]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Acre - IDH Renda,Acre - IDH,Acre - PIB Estadual,Acre - PIB Construção Civil,Acre - PIB Per Capita,Acre - PIB Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Acre - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - IDH Longevidade
0,-1.905150,-2.226009,-1.353005,2.059760,1.987344,1.924988,1.779974,0.019134,2.003355,-1.880579,...,-2.189414,-1.726894,-0.921504,-1.702629,-0.315197,-0.993854,0.113249,1.057352,-1.887036,-2.134245
1,1.111862,0.414528,-0.063550,-0.152816,0.557024,0.529880,0.870373,-0.621387,0.401153,0.111653,...,0.115936,-0.685537,-0.782267,-0.772996,-0.913786,-0.208720,1.433925,0.825941,0.092787,0.732181
2,-0.016502,0.473759,0.781607,-0.883428,-0.351137,-0.298405,-0.328802,-0.753466,-0.407217,1.154682,...,0.390533,0.050456,-0.811666,0.318328,-1.068971,-0.483531,1.125041,0.615926,1.426268,0.710739
3,0.208887,0.407067,1.180231,-0.882673,-0.636486,-0.385400,-0.896969,-0.809829,-0.830898,0.955301,...,0.311583,0.254978,-0.187878,1.329165,-0.212833,-0.715062,-0.520466,0.096038,0.537224,0.555271
4,0.974884,0.623190,0.730557,-0.357268,-0.773377,-0.563245,-0.983185,0.071542,-0.963455,0.100493,...,0.725090,0.692532,1.042762,0.796568,0.653055,0.381814,-0.876603,-0.826696,0.131381,0.275605
5,-0.373980,0.307465,-1.275841,0.216425,-0.783367,-1.207818,-0.441390,2.094006,-0.202939,-0.441549,...,0.646273,1.414466,1.660553,0.031565,1.857732,2.019354,-1.275147,-1.768562,-0.300625,-0.139550


In [19]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    151.486629
1    134.223086
2    150.491000
3    148.696000
4    153.806000
5    178.033000
Name: Acre - Consumo de Cimento (t), dtype: float64

In [20]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Acre - IDH Renda,Acre - IDH,Acre - PIB Estadual,Acre - PIB Construção Civil,Acre - PIB Per Capita,Acre - PIB Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Acre - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - IDH Longevidade
6,-0.876063,-0.179618,-2.102725,0.757798,-0.492406,-1.404695,1.16956,2.035192,0.985175,-0.705937,...,0.209576,0.780583,1.091537,0.211764,1.992786,1.69903,-1.932275,-1.663116,-0.779968,-0.628616


In [21]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    189.357
Name: Acre - Consumo de Cimento (t), dtype: float64

In [1]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, verbose=1)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [2]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [3]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

NameError: name 'train_input' is not defined

In [26]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 330ms/step - loss: 23495.2344 - val_loss: 31235.0820
Epoch 2/5000
1/1 [==============================] - 0s 27ms/step - loss: 21302.1523 - val_loss: 9456.3994
Epoch 3/5000
1/1 [==============================] - 0s 31ms/step - loss: 24188.5996 - val_loss: 41162.0938
Epoch 4/5000
1/1 [==============================] - 0s 49ms/step - loss: 18694.6992 - val_loss: 45582.5938
Epoch 5/5000
1/1 [==============================] - 0s 31ms/step - loss: 16571.7676 - val_loss: 38919.5977
Epoch 6/5000
1/1 [==============================] - 0s 26ms/step - loss: 17801.0098 - val_loss: 32509.5820
Epoch 7/5000
1/1 [==============================] - 0s 29ms/step - loss: 17051.5840 - val_loss: 18356.0488
Epoch 8/5000
1/1 [==============================] - 0s 30ms/step - loss: 19111.8613 - val_loss: 19077.8965
Epoch 9/5000
1/1 [==============================] - 0s 27ms/step - loss: 16248.6357 - val_loss: 25863.3555
Epoch 10/5000
1/1 [==================

Epoch 78/5000
1/1 [==============================] - 0s 28ms/step - loss: 1693.9808 - val_loss: 3001.6975
Epoch 79/5000
1/1 [==============================] - 0s 27ms/step - loss: 1632.8177 - val_loss: 2857.3425
Epoch 80/5000
1/1 [==============================] - 0s 27ms/step - loss: 1851.5626 - val_loss: 2860.2488
Epoch 81/5000
1/1 [==============================] - 0s 26ms/step - loss: 1531.4443 - val_loss: 2558.1082
Epoch 82/5000
1/1 [==============================] - 0s 26ms/step - loss: 1480.2245 - val_loss: 2439.6931
Epoch 83/5000
1/1 [==============================] - 0s 27ms/step - loss: 1434.2894 - val_loss: 2343.2817
Epoch 84/5000
1/1 [==============================] - 0s 29ms/step - loss: 1344.6191 - val_loss: 2325.3877
Epoch 85/5000
1/1 [==============================] - 0s 30ms/step - loss: 1669.0948 - val_loss: 3159.8943
Epoch 86/5000
1/1 [==============================] - 0s 27ms/step - loss: 1519.4253 - val_loss: 5744.5474
Epoch 87/5000
1/1 [===========================

1/1 [==============================] - 0s 30ms/step - loss: 20.5125 - val_loss: 65.1615
Epoch 234/5000
1/1 [==============================] - 0s 28ms/step - loss: 20.0361 - val_loss: 63.8373
Epoch 235/5000
1/1 [==============================] - 0s 25ms/step - loss: 19.5686 - val_loss: 62.4880
Epoch 236/5000
1/1 [==============================] - 0s 26ms/step - loss: 19.1340 - val_loss: 61.1635
Epoch 237/5000
1/1 [==============================] - 0s 25ms/step - loss: 18.7121 - val_loss: 59.8562
Epoch 238/5000
1/1 [==============================] - 0s 26ms/step - loss: 18.3068 - val_loss: 58.5292
Epoch 239/5000
1/1 [==============================] - 0s 28ms/step - loss: 17.9142 - val_loss: 57.2392
Epoch 240/5000
1/1 [==============================] - 0s 30ms/step - loss: 17.5347 - val_loss: 55.9684
Epoch 241/5000
1/1 [==============================] - 0s 29ms/step - loss: 17.1733 - val_loss: 54.7287
Epoch 242/5000
1/1 [==============================] - 0s 26ms/step - loss: 16.8220 - val

1/1 [==============================] - 0s 27ms/step - loss: 7.9620 - val_loss: 16.9989
Epoch 314/5000
1/1 [==============================] - 0s 27ms/step - loss: 7.9251 - val_loss: 16.8211
Epoch 315/5000
1/1 [==============================] - 0s 25ms/step - loss: 7.8894 - val_loss: 16.6427
Epoch 316/5000
1/1 [==============================] - 0s 28ms/step - loss: 7.8553 - val_loss: 16.4645
Epoch 317/5000
1/1 [==============================] - 0s 27ms/step - loss: 7.8211 - val_loss: 16.2875
Epoch 318/5000
1/1 [==============================] - 0s 26ms/step - loss: 7.7892 - val_loss: 16.1131
Epoch 319/5000
1/1 [==============================] - 0s 26ms/step - loss: 7.7576 - val_loss: 15.9405
Epoch 320/5000
1/1 [==============================] - 0s 26ms/step - loss: 7.7272 - val_loss: 15.7700
Epoch 321/5000
1/1 [==============================] - 0s 30ms/step - loss: 7.6974 - val_loss: 15.6021
Epoch 322/5000
1/1 [==============================] - 0s 30ms/step - loss: 7.6688 - val_loss: 15.

1/1 [==============================] - 0s 29ms/step - loss: 6.5527 - val_loss: 8.1811
Epoch 475/5000
1/1 [==============================] - 0s 29ms/step - loss: 6.5513 - val_loss: 8.1697
Epoch 476/5000
1/1 [==============================] - 0s 26ms/step - loss: 6.5494 - val_loss: 8.1562
Epoch 477/5000
1/1 [==============================] - 0s 28ms/step - loss: 6.5494 - val_loss: 8.1461
Epoch 478/5000
1/1 [==============================] - 0s 27ms/step - loss: 6.5486 - val_loss: 8.1345
Epoch 479/5000
1/1 [==============================] - 0s 27ms/step - loss: 6.5477 - val_loss: 8.1234
Epoch 480/5000
1/1 [==============================] - 0s 26ms/step - loss: 6.6084 - val_loss: 8.1520
Epoch 481/5000
1/1 [==============================] - 0s 26ms/step - loss: 6.5461 - val_loss: 8.0919
Epoch 482/5000
1/1 [==============================] - 0s 30ms/step - loss: 6.5452 - val_loss: 8.0870
Epoch 483/5000
1/1 [==============================] - 0s 28ms/step - loss: 6.5443 - val_loss: 8.0771
Epoch

1/1 [==============================] - 0s 34ms/step - loss: 6.4973 - val_loss: 7.5040
Epoch 637/5000
1/1 [==============================] - 0s 28ms/step - loss: 6.4967 - val_loss: 7.5035
Epoch 638/5000
1/1 [==============================] - 0s 26ms/step - loss: 6.4966 - val_loss: 7.5024
Epoch 639/5000
1/1 [==============================] - 0s 26ms/step - loss: 6.4965 - val_loss: 7.5014
Epoch 640/5000
1/1 [==============================] - 0s 27ms/step - loss: 6.4965 - val_loss: 7.5005
Epoch 641/5000
1/1 [==============================] - 0s 27ms/step - loss: 6.4964 - val_loss: 7.4995
Epoch 642/5000
1/1 [==============================] - 0s 26ms/step - loss: 6.4963 - val_loss: 7.4986
Epoch 643/5000
1/1 [==============================] - 0s 28ms/step - loss: 6.4962 - val_loss: 7.4978
Epoch 644/5000
1/1 [==============================] - 0s 25ms/step - loss: 6.4961 - val_loss: 7.4968
Epoch 645/5000
1/1 [==============================] - 0s 28ms/step - loss: 6.4961 - val_loss: 7.4958
Epoch

1/1 [==============================] - 0s 27ms/step - loss: 6.4910 - val_loss: 7.4546
Epoch 799/5000
1/1 [==============================] - 0s 27ms/step - loss: 6.4910 - val_loss: 7.4539
Epoch 800/5000
1/1 [==============================] - 0s 30ms/step - loss: 6.4910 - val_loss: 7.4531
Epoch 801/5000
1/1 [==============================] - 0s 28ms/step - loss: 78.6324 - val_loss: 1.1879
Epoch 802/5000
1/1 [==============================] - 0s 29ms/step - loss: 100.2557 - val_loss: 42.3242
Epoch 803/5000
1/1 [==============================] - 0s 27ms/step - loss: 55.3497 - val_loss: 39.5018
Epoch 804/5000
1/1 [==============================] - 0s 24ms/step - loss: 51.6806 - val_loss: 45.4075
Epoch 805/5000
1/1 [==============================] - 0s 28ms/step - loss: 90.4570 - val_loss: 18.5723
Epoch 806/5000
1/1 [==============================] - 0s 25ms/step - loss: 55.4491 - val_loss: 54.9075
Epoch 807/5000
1/1 [==============================] - 0s 26ms/step - loss: 47.6064 - val_loss:

In [27]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 46ms/step
[189.357](test_target) - [[171.79665]](prediction) = 17.560353881835937


In [28]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [29]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Acre - IDH Renda,Acre - IDH,Acre - PIB Estadual,Acre - PIB Construção Civil,Acre - PIB Per Capita,Acre - PIB Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Acre - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - IDH Longevidade
0,-1.313791,-1.413953,-1.300362,1.373448,1.305102,1.314658,1.165029,1.396038,1.335266,-1.330895,...,-1.407559,-1.288562,-1.385554,-1.190812,1.387082,-1.327445,-1.378418,1.244046,-1.296189,-1.414184
1,1.110178,0.683452,0.168731,-0.394781,-0.180819,-0.205935,0.111769,-0.502296,-0.264151,0.251276,...,0.585103,0.139596,0.938093,-0.065256,-0.454788,1.086118,0.963021,-0.039560,0.158277,0.715055
2,0.203613,0.730500,1.131631,-0.978667,-1.124283,-1.108722,-1.276798,-0.893743,-1.071116,1.079619,...,0.822455,1.148965,0.447461,1.256068,-0.932294,0.241327,0.415397,-1.204486,1.137912,0.699128


train_target:


,Acre - Consumo de Cimento (t)
0,151.486629
1,134.223086
2,150.491000


test_input:


...

,Acre - IDH Renda,Acre - IDH,Acre - PIB Estadual,Acre - PIB Construção Civil,Acre - PIB Per Capita,Acre - PIB Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Acre - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - IDH Longevidade
6,-0.876063,-0.179618,-2.102725,0.757798,-0.492406,-1.404695,1.16956,2.035192,0.985175,-0.705937,...,0.209576,0.780583,1.091537,0.211764,1.992786,1.69903,-1.932275,-1.663116,-0.779968,-0.628616


test_target:


,Acre - Consumo de Cimento (t)
6,189.357


Epoch 922: early stopping
1/1 [==============================] - 0s 34ms/step
Error: 17.560353881835937




[142.3567657470703, 149.59434509277344, 151.25064086914062, 171.79664611816406]

In [30]:
display(targets)
display(predictions)

[148.696, 153.80599999999998, 178.033, 189.357]

[142.3567657470703, 149.59434509277344, 151.25064086914062, 171.79664611816406]

In [31]:
mae = mean_absolute_error(predictions, targets)
mae

13.723400543212882

In [32]:
porcentage = mae/np.mean(targets)
porcentage

0.0819439583885933